In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/MasterThesis/SplittedImagesAllClasses'):
    for filename in filenames:
        os.path.join(dirname, filename)

In [2]:
!git clone https://github.com/joerivc/MasterThesis.git

Cloning into 'MasterThesis'...
remote: Enumerating objects: 6654, done.
remote: Total 6654 (delta 0), reused 0 (delta 0), pack-reused 6654
Receiving objects: 100% (6654/6654), 1.21 GiB | 42.43 MiB/s, done.
Resolving deltas: 100% (107/107), done.
Updating files: 100% (21222/21222), done.


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
DATADIRTRAIN = '/content/MasterThesis/SplittedImagesAllClasses/train'
DATADIRTEST = '/content/MasterThesis/SplittedImagesAllClasses/test'
DATADIRVAL = '/content/MasterThesis/SplittedImagesAllClasses/val'
CATEGORIES = ['Sitting','SittingPelvicRF','SittingMPelvicRF','SittingSPelvicRF','SittingPelvicRB','SittingSPelvicRB','SittingMPelvicRB','Standing','StandingPelvicRF','StandingSPelvicRF','StandingMPelvicRF','StandingMPelvicRB','StandingSPelvicRB','StandingPelvicRB']
IMG_SIZE=299

In [5]:
training_data=[]
def create_training_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIRTRAIN, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                training_data.append([new_array,class_num])
            except Exception as e:
                pass

In [6]:
test_data=[]
def create_test_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIRTEST, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                test_data.append([new_array,class_num])
            except Exception as e:
                pass

In [7]:
val_data=[]
def create_val_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIRVAL, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                val_data.append([new_array,class_num])
            except Exception as e:
                pass

In [8]:
create_training_data()
X_train=[]
y_train=[]
categories_train = []
label_train = []
for categories_train, label_train in training_data:
    X_train.append(categories_train)
    y_train.append(label_train)


In [9]:
X_train1 = np.array(X_train)/255.0
y_train = np.array(y_train)

In [10]:
create_test_data()
X_test=[]
y_test=[]
for categories_test, label_test in test_data:
    X_test.append(categories_test)
    y_test.append(label_test)

In [11]:
X_test1 = np.array(X_test)/255.0

In [12]:
create_val_data()
X_val=[]
y_val=[]
for categories_val, label_val in val_data:
    X_val.append(categories_val)
    y_val.append(label_val)

In [13]:
X_val1 = np.array(X_val)/255.0
y_val = np.array(y_val)

In [14]:
print(X_train1.shape)

(2880, 299, 299, 3)


In [15]:
print(len(y_train))

2880


In [16]:
X_test = np.array(X_test)/255.0
y_test = np.array(y_test)

In [17]:
# Load pretrained Xception model
xception = tf.keras.applications.Xception(weights='imagenet', include_top=True)
xception.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#train_datagen = ImageDataGenerator()
#train_datagen.fit(X_train)
#xception.fit(train_datagen.flow(X_train, y_train, batch_size=32),
#                    steps_per_epoch=len(x_train) / batch_size, epochs=10, verbose=1)
print(X_train1.shape)
xception.fit(X_train1, y_train,epochs=20,batch_size=32,validation_data=(X_val1, y_val))

91884032/91884032 [==============================] - 1s 0us/step
(2880, 299, 299, 3)
Epoch 1/20
90/90 [==============================] - 77s 464ms/step - loss: 1.5139 - accuracy: 0.4583 - val_loss: 8.1766 - val_accuracy: 0.2792
Epoch 2/20
90/90 [==============================] - 38s 417ms/step - loss: 0.7685 - accuracy: 0.6771 - val_loss: 4.5625 - val_accuracy: 0.4500
Epoch 3/20
90/90 [==============================] - 38s 423ms/step - loss: 0.5250 - accuracy: 0.7969 - val_loss: 1.4005 - val_accuracy: 0.6896
Epoch 4/20
90/90 [==============================] - 39s 428ms/step - loss: 0.3642 - accuracy: 0.8681 - val_loss: 1.7250 - val_accuracy: 0.6427
Epoch 5/20
90/90 [==============================] - 39s 430ms/step - loss: 0.2976 - accuracy: 0.8892 - val_loss: 0.6353 - val_accuracy: 0.8115
Epoch 6/20
90/90 [==============================] - 39s 429ms/step - loss: 0.2198 - accuracy: 0.9191 - val_loss: 0.4020 - val_accuracy: 0.8521
Epoch 7/20
90/90 [==============================] - 39s 4

In [18]:
loss, accuracy = xception.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

30/30 [==============================] - 4s 127ms/step - loss: 0.6369 - accuracy: 0.8479
Test Loss: 0.6368528604507446, Test Accuracy: 0.8479166626930237
